# Hamiltonian replica exchange simulation (REST3) using GROMACS


# Background

Compared to conventional molecular dynamics (MD) simulations, the replica exchange molecular dynamics (REMD) simulations combines multiple parallel simulations with different conditions, which normally can accelerate the conformational sampling, given that these parallel simulations can sample different states of interest. A representative example is the Hamiltonian replica exchange (HREX) simulation, where we can define different biased potentials for the solute, but keep the solvent untouched. This can save many replicas to use. In this note, I will introduce how we can use the GROMACS program to run the HREX simulations (such as REST3). The readers can also check out one paper and future REST3 paper for the details [1,2].

<img src='https://www.mdpi.com/biomolecules/biomolecules-11-01416/article_deploy/html/images/biomolecules-11-01416-g002.png' alt='drawing' width='400'/>

**References**

[1]. Bussi, Hamiltonian replica exchange in GROMACS: a flexible implementation, Molecular Physics 2013, 112, 379-384. [DOI](https://doi.org/10.1080/00268976.2013.824126)

[2]. Yumeng Zhang, Xiaorong Liu, and Jianhan Chen, Re-balancing Replica Exchange with Solute Tempering for Sampling Dynamic Protein Conformations, JCTC, 19, 1602-1614 (2023) [DOI](https://pubs.acs.org/doi/10.1021/acs.jctc.2c01139)

# How to run REST3 simulation

The GROMACS program can be used to perform the HREX simulation by patching with the PLUMED. The basic idea is that we can create different topologies for all conditions, and then this HREX simulation can be enabled by using the -hrex flag of mdrun. The usage is almost similar to the conventional MD simulations. The REST3 simulation is one of HREX simulations, which has the following equation,

$E_{m}^{REST3}(r) = \lambda_{m}^{pp} E_{pp}(r) +
\lambda_{m}^{pw} E_{pw}^{elec}(r) + 
\kappa_m \lambda_{m}^{pw} E_{pw}^{vdW}(r) +
\lambda^{ww} E_{ww}(r)$, (1)

where the $m$ will loop all replicas, and they can have different parameters, including the $\lambda_{m}^{pp}, \lambda_{m}^{pw}, \kappa_m$ .

**Notes:**

This tutorial does **NOT** contain any introductions to the basic usage of GROMACS. 

The users are assumed to be famililar with using GROMACS to run the MD simulations. For example, you know how to create the simulation system of interest, do the energy minimizations, equilibrium and production steps. The users are also encouraged to [gromacs_documentation](https://manual.gromacs.org/current/user-guide/index.html) for a pre-studying.


## Step 1: create systems

First, you need to know how many replicas/conditions you will use in the HREX simulation, then we should create an initial simulation state for each condition. In princple, you can create different initial states for different conditions, but also you can use the same initial state for all conditions. However, if you use the same states, then we probably need more simulation time to make systems be equilibrated. This step is the same as the MD simulation. 

In the end, you will create two files, which can be: 

**prot.gro** 

and 

**prot.top**.


## Step 2: create tpr files for all replicas/conditions

The [tpr file](https://manual.gromacs.org/archive/5.0.4/online/tpr.html) has all simulation parameters and is ready to run the MD production. However, different conditions could have different topology parameters, so we have to create each tpr file for each condition. 
Given that the conditions in the REST3 have different scaled parameters (Eq. 2), The PLUMED provided a script named "partial_tempering", which can generate scaled topologies. Please also refer to [the notes from PLUMED](https://www.plumed.org/doc-v2.7/user-doc/html/hrex.html). The followings are what I used to create the tpr file.

---------------------------

**1. Obtain 'process.top' file for later partial_tempering.**

First, we can use the "gmx grompp" to generate the "processed.top" file, which is an independent topology file and includes all parameters. 

> Bash script example

```bash
    # work directory: 
    gmx=/home/zgjia/Software/gromacs/gromacs514GPU_plumed/bin/gmx_mpi
    
    # generate processed*.top
    # we need to prepare several files: rest.mdp, prot.gro, prot.top, etc.
    # These can be found in the setup directory
    $gmx grompp -f rest.mdp -c prot.gro -p prot.top -pp processed.top
    
    exit
```
**Extra steps before moving on:** 

before commenting the following exit: modify the processed.top, by doing this, we can:
    
1.1 Define the hot atoms that must have a "_" appended to the atom type,
   please refer to the link: https://www.plumed.org/doc-v2.7/user-doc/html/hrex.html
   
```
i.e., 

before selection:
; residue   1 ACE rtp ACE  q  0.0
     1         CT      1    ACE    CH3      1    -0.3662      12.01   ; qtot -0.3662

after selection:
; residue   1 ACE rtp ACE  q  0.0
     1         CT_     1    ACE    CH3      1    -0.3662      12.01   ; qtot -0.3662
```

1.2 Remove the unnecessary the lines [ nonbond_params ] if any,
   here, this modified c36mr-tip4pd force field will generate two lines with the [ nonbond_params ],
   just delete the first one, which is unnecessary. If it only has one, then please ignore this step.
   
   
--------------------------------------

**2. Executing REST3 scaling.**

Then, after seleting the 'hot atoms', we are able to scale the topology file based on REST3 algorithm.

For parameters and algorithm, please refer to the [REST3 paper](https://pubs.acs.org/doi/10.1021/acs.jctc.2c01139).

(below is an example for REST3 8-replica scaling spanning from 298 - 450 K.)

> Bash script example

```bash   
    # scaled topology files using REST3 paprameters (8 rep as an example)
    # two variable after bash scripts are $\lambda_{m}^{pp}$ and $\kappa_m \lambda_{m}^{pw}$, respectively
    bash gennewtop.sh 1.000 1.000 processed.top topol0.top
    bash gennewtop.sh 0.943 1.000 processed.top topol1.top
    bash gennewtop.sh 0.889 1.003 processed.top topol2.top
    bash gennewtop.sh 0.838 1.010 processed.top topol3.top
    bash gennewtop.sh 0.790 1.020 processed.top topol4.top
    bash gennewtop.sh 0.745 1.027 processed.top topol5.top
    bash gennewtop.sh 0.702 1.035 processed.top topol6.top
    bash gennewtop.sh 0.662 1.045 processed.top topol7.top
    
    # create tpr files: topol[0-7].top (Temperature ranged 298 - 450 K)
    $gmx grompp -maxwarn 3 -o topol0.tpr -f rest.mdp -p topol0.top -c npt.gro -r npt.gro
    $gmx grompp -maxwarn 3 -o topol1.tpr -f rest.mdp -p topol1.top -c npt.gro -r npt.gro
    $gmx grompp -maxwarn 3 -o topol2.tpr -f rest.mdp -p topol2.top -c npt.gro -r npt.gro
    $gmx grompp -maxwarn 3 -o topol3.tpr -f rest.mdp -p topol3.top -c npt.gro -r npt.gro
    $gmx grompp -maxwarn 3 -o topol4.tpr -f rest.mdp -p topol4.top -c npt.gro -r npt.gro
    $gmx grompp -maxwarn 3 -o topol5.tpr -f rest.mdp -p topol5.top -c npt.gro -r npt.gro
    $gmx grompp -maxwarn 3 -o topol6.tpr -f rest.mdp -p topol6.top -c npt.gro -r npt.gro
    $gmx grompp -maxwarn 3 -o topol7.tpr -f rest.mdp -p topol7.top -c npt.gro -r npt.gro
    
    rm files with the comments
    rm \#*
    
    exit

```

## Step 3: run REST3 simulation

Create directories for all replica simulations, and copy these tpr files into them.

> Bash script example
```bash
    cd /home/work/ # the main directory to run the REST3 simulation
    mkdir -p rep0; cp setup/topol0.tpr rep0/topol.tpr
    mkdir -p rep1; cp setup/topol1.tpr rep1/topol.tpr
    mkdir -p rep2; cp setup/topol2.tpr rep2/topol.tpr
    mkdir -p rep3; cp setup/topol3.tpr rep3/topol.tpr
    mkdir -p rep4; cp setup/topol4.tpr rep4/topol.tpr
    mkdir -p rep5; cp setup/topol5.tpr rep5/topol.tpr
    mkdir -p rep6; cp setup/topol6.tpr rep6/topol.tpr
    mkdir -p rep7; cp setup/topol7.tpr rep7/topol.tpr
    
    # check out the bash script "run-local-first.slurm" to run the job.

```

# Appendix
